In [1]:
import os
import torch

# Define the MNIST CNN model architecture (as used during training)
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 20, 5, 1)
        self.conv2 = torch.nn.Conv2d(20, 50, 5, 1)
        self.fc1 = torch.nn.Linear(4 * 4 * 50, 500)
        self.fc2 = torch.nn.Linear(500, 10)

    def forward(self, x):
        import torch.nn.functional as F
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
image_path="/Users/abdhumal/Dev/RedHatDev/k8s-ml-lab/examples/01-complete-workflow/test_images/digit3.jpg"
model_path="/Users/abdhumal/Dev/RedHatDev/k8s-ml-lab/output/mnist_model.pth"
state_dict = torch.load(model_path, map_location=device, weights_only=True)
model.load_state_dict(state_dict)
model.eval()
print("Model loaded and set to evaluation mode.")

Model loaded and set to evaluation mode.


In [4]:
import torchvision.transforms as transforms
from PIL import Image

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the image and apply the transformations
image = Image.open(image_path)
image = transform(image).unsqueeze(0).to(device)

# Perform inference using the model
with torch.no_grad():
    output = model(image)
    predicted = output.argmax(dim=1, keepdim=True).item()

print(f"Predicted handwritten digit: {predicted}")


Predicted handwritten digit: 3
